First we need to import some functions

In [1]:
import pandas as pd, numpy as np
import datetime, time, os, pylast
#We may not need pylast

Once we have a run the processing script we can do some analysis with it. First load script.

In [2]:
filename=r"D:\Coding\GitHub\lastFMToolsv2\AllScrobblesTo_20250107_141214.csv"
dateparse = '%Y-%m-%d %H:%M:%S'
df = pd.read_csv(filename, date_format=dateparse)
df['Scrobble Time']=pd.to_datetime(df['Scrobble Time'])
df.head()

,Scrobble Time,Track,Artist,Album,Album Artist,Year
0,2010-12-23 15:35:29,Take Me Back To The Islands,Idlewild,Post Electric Blues,Idlewild,2010
1,2010-12-23 15:40:28,You Held the World in Your Arms,Idlewild,The Remote Part,Idlewild,2010
2,2010-12-23 15:44:59,Maps,Yeah Yeah Yeahs,Fever To Tell,Yeah Yeah Yeahs,2010
3,2010-12-23 15:49:39,Sweet Disposition,The Temper Trap,Conditions,The Temper Trap,2010
4,2010-12-23 15:53:54,Hot n Cold,Katy Perry,One of the Boys,Katy Perry,2010


We now want to pivot the table to get annualised counts for each year


In [3]:
keys=["Artist"]
sortingLogic=[False,True]
# if fieldtype in ["Track","Album"]:
#     keys.append(fieldtype)
#     sortingLogic=[False,True,True]


        
    
        
s=(df.pivot_table(index=keys,columns="Year",aggfunc=['size'],sort=True).fillna(0).astype(int))
s=s.reset_index()

s.columns = [f'{i}' if i != '' and j=='' else f'{j}' for i,j in s.columns]

s['Total']=s[s.columns.to_list()[2:]].sum(axis=1)

sortKeys=['Total']+keys
s=s.sort_values(sortKeys,ascending=sortingLogic)

s.reset_index(inplace=True,drop=True)
s.head()
s.loc[s["Artist"]=="Måneskin"]

,Artist,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,Total
115,Måneskin,0,0,0,0,0,0,0,0,0,0,0,0,12,131,33,0,176


In [4]:
n=1
for k in range(2010,2025):
    s["R"+str(k)] = s.groupby('Artist')[str(k)].transform(np.sum).rank(ascending=False, method='min').astype(int)
    
    if k==2010:
        t=s[["R"+str(k),"Artist",str(k)]].sort_values("R"+str(k),ascending=True)
        t.columns=["Pos","Artist","Count"]
    else:
        n=n+1
        s["Diff"]=s["R"+str(k-1)]-s["R"+str(k)]
        s["SUM"]=s.iloc[:, 2:n].sum(axis=1)
        s["DiffS"]=s["Diff"].astype(str)
        s["RA"+str(k-1)]=s["R"+str(k-1)].astype(str)
        j=s["R"+str(k-1)].max()
        s.loc[s["R"+str(k-1)]==j,"RA"+str(k-1)]="-"
        s.loc[s["R"+str(k-1)]==j,"DiffS"]="NE"
        s.loc[(s["R"+str(k-1)]==j) & (s["SUM"]>0),"DiffS"]="RE"
        t=s[["R"+str(k),"RA"+str(k-1),"DiffS","Artist",str(k)]].sort_values("R"+str(k),ascending=True)

        t.columns=["Pos","Last Year","Change","Artist","Count"]
    t.to_csv(str(k)+"_ArtistChart.csv",index=False)
    


Below this should be the to date counts

In [5]:
for k in range(2010,2025):

    
    
    if k==2010:
        s[str(k)+"CumCount"]=s[str(k)]
    else:
        s[str(k)+"CumCount"]=s[str(k)]+s[str(k-1)+"CumCount"]

s

,Artist,2010,2011,2012,2013,2014,2015,2016,2017,2018,...,2015CumCount,2016CumCount,2017CumCount,2018CumCount,2019CumCount,2020CumCount,2021CumCount,2022CumCount,2023CumCount,2024CumCount
0,Muse,8,89,203,178,85,291,82,50,179,...,854,936,986,1165,1195,1210,1259,1437,1723,1941
1,Ash,12,110,112,135,82,143,68,51,279,...,594,662,713,992,1074,1120,1127,1211,1566,1869
2,Feeder,11,83,79,157,102,142,68,80,140,...,574,642,722,862,952,960,984,1042,1254,1694
3,Queen,3,8,70,74,25,30,15,32,347,...,210,225,257,604,720,730,795,858,1181,1431
4,Frank Turner,6,119,54,148,82,100,34,24,57,...,509,543,567,624,634,638,685,740,962,1361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4504,songer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4505,stellastarr*,0,0,0,0,0,1,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4506,venbee & Goddard.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4507,Şebnem Ferah,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [6]:
n=1
for k in range(2010,2025):
    s[str(k)+"CumRank"] = s.groupby('Artist')[str(k)+"CumCount"].transform(np.sum).rank(ascending=False, method='min').astype(int)
    if k==2010:
        t=s[[str(k)+"CumRank","Artist",str(k)+"CumCount"]].sort_values(str(k)+"CumRank",ascending=True)
        t.columns=["Pos","Artist","CumulativeCount"]
    else:
        n=n+1
        s["Diff"]=s[str(k-1)+"CumRank"]-s[str(k)+"CumRank"]
        s["SUM"]=s.iloc[:, 2:n].sum(axis=1)
        s["DiffS"]=s["Diff"].astype(str)
        s[str(k-1)+"CumRankLast"]=s[str(k-1)+"CumRank"].astype(str)
        j=s[str(k-1)+"CumRank"].max()
        s.loc[s[str(k-1)+"CumRank"]==j,str(k-1)+"CumRankLast"]="-"
        s.loc[s[str(k-1)+"CumRank"]==j,"DiffS"]="NE"
        s.loc[(s[str(k-1)+"CumRank"]==j) & (s["SUM"]>0),"DiffS"]="RE"
        t=s[[str(k)+"CumRank",str(k-1)+"CumRankLast","DiffS","Artist",str(k)+"CumCount"]].sort_values(str(k)+"CumRank",ascending=True)

        t.columns=["Pos","Last Year","Change","Artist","Count"]
    t.to_csv(str(k)+"_ArtistChartCum.csv",index=False)